In [1]:
from random import uniform, randint
from typing import List, Dict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.constraints import max_norm
import matplotlib.pyplot as plt

In [2]:
COLUMNS = 3
SAMPLES = 5000

In [3]:
base = [[randint(0,5) for _ in range(SAMPLES)] for _ in range(COLUMNS)]
res = []
zeros = 0
ones = 0
twos = 0
threes = 0
for i in range(len(base[0])):
    sum = 0
    for j in range(len(base) - 1):
        sum += base[j][i]
    sum *= base[-1][i]
    if sum < 8:
        res.append(0)
        zeros += 1
    elif sum >= 8 and sum < 16.5:
        res.append(1)
        ones += 1
    elif sum >= 16.5 and sum < 27.5:
        res.append(2)
        twos += 1
    else:
        res.append(3)
        threes += 1
print(zeros)
print(ones)
print(twos)
print(threes)

2047
1460
895
598


In [4]:
noise = [uniform(0,50) for _ in range(SAMPLES)]

In [5]:
a = ord("a")
data = {chr(i): base[j] for i in range(a, a + len(base)) for j in range(len(base))}
data["noise"] = noise
data["result"] = res


In [6]:
a = ord("a")
keys = [chr(i)for i in range(a, a + len(base))]
data = {}
for key, samples in zip(keys, base):
    data[key] = samples
data["noise"] = noise
data["result"] = res

In [7]:
df = pd.DataFrame(data)
df.head(5)

,a,b,c,noise,result
0,2,5,2,21.288967,1
1,0,4,2,40.191227,1
2,0,4,5,24.984966,2
3,0,5,0,7.125543,0
4,0,1,2,20.246621,0


In [8]:
all_features = df.drop("result",axis=1)
Targeted_feature = df["result"]
X_train,X_test,y_train,y_test = train_test_split(all_features,Targeted_feature,test_size=0.3,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3500, 4), (1500, 4), (3500,), (1500,))

In [9]:
FIRST_LAYER = 10
model = Sequential()
model.add(Dense(FIRST_LAYER, activation='relu'))
model.add(Dense(10, activation='relu'))
# model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



2022-11-02 23:12:15.443543: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-02 23:12:15.443683: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
def create_history(columns: int, nodes: int) -> Dict:
    return {i:{j: [] for j in range(nodes)} for i in range(columns)}

In [11]:
history = create_history(len(df.columns), FIRST_LAYER)
history

{0: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []},
 1: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []},
 2: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []},
 3: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []},
 4: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}}

In [12]:
for epoch in range(1, 501):
    print(f"RUNNING EPOCH {epoch}")
    hist = model.fit(
      x=X_train, 
      y=y_train, 
      epochs=1,
      batch_size=100,
    )
    all_weights = model.layers[0].get_weights()[0].tolist()
    for column in range(len(all_weights)):
        for node in range(len(all_weights[column])):
            history[column][node].append(all_weights[column][node])

RUNNING EPOCH 1


2022-11-02 23:12:17.909117: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1/35 [..............................] - ETA: 13s - loss: -1.6018 - accuracy: 0.2700

2022-11-02 23:12:18.162374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 1s 6ms/step - loss: -0.3935 - accuracy: 0.2923
RUNNING EPOCH 2
35/35 [==============================] - 0s 6ms/step - loss: -0.7170 - accuracy: 0.2920
RUNNING EPOCH 3
35/35 [==============================] - 0s 6ms/step - loss: -1.0950 - accuracy: 0.2920
RUNNING EPOCH 4
35/35 [==============================] - 0s 6ms/step - loss: -1.6084 - accuracy: 0.2920
RUNNING EPOCH 5
35/35 [==============================] - 0s 6ms/step - loss: -2.2131 - accuracy: 0.2920
RUNNING EPOCH 6
35/35 [==============================] - 0s 6ms/step - loss: -2.9622 - accuracy: 0.2920
RUNNING EPOCH 7
35/35 [==============================] - 0s 6ms/step - loss: -3.8782 - accuracy: 0.2920
RUNNING EPOCH 8
35/35 [==============================] - 0s 6ms/step - loss: -5.0556 - accuracy: 0.2920
RUNNING EPOCH 9
35/35 [==============================] - 0s 6ms/step - loss: -6.6084 - accuracy: 0.2926
RUNNING EPOCH 10
35/35 [==============================] - 0s 6ms/step - loss: -9

KeyboardInterrupt: 